In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Exercise 08 - Monte Carlo simulation

#### 8.1) Lennard-Jones Potential

#### 8.2) 2D-potential

First we show the potential landscape and the corresponding distribution. Afterwards we will answers the questions.

For this problem, since the units of the potential were not given, we assume that it has units of $k_{B}T$. For the final question, we will consider different temperature values by varying a coefficient.

In [ ]:
def U(x, y):
    return np.square(x**2 - 1) + (x - y) + y**2

def uu(xx):
    """Wrapper for an np.ndarray. xx[..., 0] = x, xx[..., 1] = y"""
    return U(xx[..., 0], xx[..., 1])

In [ ]:
endpt = 2.5
numpt = 100

xx = np.linspace(-endpt, endpt, numpt)
yy = np.linspace(-endpt, endpt, numpt)
x, y = np.meshgrid(xx, yy)
z = U(x, y)

fig, ax = plt.subplots()
ax.contourf(x, y, z, levels=20)
ax.set_title("Contour plot of Energy Landscape")
ax.set_xlabel("x")
ax.set_ylabel("y")
plt.show()

In [ ]:
def Boltzmann(x, y, beta=1.0):
    return np.exp(-beta * U(x, y))

def boltzmann(xx, beta=1.0):
    """Wrapper for an np.ndarray. xx[..., 0] = x, xx[..., 1] = y"""
    return Boltzmann(xx[..., 0], xx[..., 1], beta=1.0)

In [ ]:
z = Boltzmann(x, y, beta=1.0)

fig, ax = plt.subplots()
ax.contourf(x, y, z, levels=20)
ax.set_title("Boltzmann Distribution")
ax.set_xlabel("x")
ax.set_ylabel("y")
plt.show()

Metropolis MCMC for sampling.

In [ ]:
def metropolis(state, step_size=1.0, beta=1.0):
    step = (2 * np.random.rand(*state.shape) - 1) * step_size
    proposal = state + step
    diff = uu(proposal) - uu(state)
    if diff <= 0 or (np.random.rand() < np.exp(-beta * diff)):
        return proposal
    else:
        return state
    
def mcmc(initial_state, steps, step_size=1.0, beta=1.0):
    states = [initial_state]
    for _ in range(steps):
        state = metropolis(states[-1], step_size=step_size, beta=beta)
        states.append(state)
    return np.asarray(states)

In [ ]:
states = mcmc(np.array([0.0, 0.0]), 10000)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(*states.T)
ax.set_title("Samples from MCMC")
ax.set_xlabel("x")
ax.set_ylabel("y")
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.hist2d(*states.T, bins=30, normed=True, range=((-endpt, endpt), (-endpt, endpt)))
ax.set_title("Histogram from MCMC")
ax.set_xlabel("x")
ax.set_ylabel("y")
plt.show()

Step size and effect on acceptance ratio. As step size decreases, we accept more proposals; however, we do not get good "mixing." Mixing colloquially means visiting a significant portion of the distribution with high regularity.

In [ ]:
def acceptance(samples):
    accepted = []
    for i in range(len(samples)):
        try:
            if np.all(np.equal(samples[i], samples[i+1])):
                accepted.append(False)
            else:
                accepted.append(True)
        except IndexError:
            break
    return accepted

def acceptance_ratio(samples):
    arr = acceptance(samples)
    return sum(arr)/len(arr)

In [ ]:
ar = []
delta = np.linspace(0.01, 1.0, 25)
for d in delta:
    smps = mcmc(np.array([0.0, 0.0]), 10000, step_size=d)
    ar.append(acceptance_ratio(smps))

In [ ]:
fig, ax = plt.subplots()
ax.plot(delta, ar)
ax.set_title("Acceptance Ratio versus Step Size")
ax.set_xlabel("Delta")
ax.set_ylabel("Acceptance Ratio")
plt.show()

We alter the temperature by altering the $\beta$ coefficient. As stated above, there are no units for this energy, so we select $\beta$ between 1.0 and 0.25. This represents a temperature change of 4 times the initial temperature. We have already calculated $\beta$=1.0 above. 

In [ ]:
beta = 0.75
smps = mcmc(np.array([0.0, 0.0]), 10000, step_size=1.0, beta=beta)

In [ ]:
fig, ax = plt.subplots()
ax.hist2d(*smps.T, bins=30, normed=True, range=((-endpt, endpt), (-endpt, endpt)))
ax.set_title(f"Histogram from MCMC with beta={beta}")
ax.set_xlabel("x")
ax.set_ylabel("y")
plt.show()

In [ ]:
beta = 0.5
smps = mcmc(np.array([0.0, 0.0]), 10000, step_size=1.0, beta=beta)

In [ ]:
fig, ax = plt.subplots()
ax.hist2d(*smps.T, bins=30, normed=True, range=((-endpt, endpt), (-endpt, endpt)))
ax.set_title(f"Histogram from MCMC with beta={beta}")
ax.set_xlabel("x")
ax.set_ylabel("y")
plt.show()

In [ ]:
beta = 0.25
smps = mcmc(np.array([0.0, 0.0]), 10000, step_size=1.0, beta=beta)

In [ ]:
fig, ax = plt.subplots()
ax.hist2d(*smps.T, bins=30, normed=True, range=((-endpt, endpt), (-endpt, endpt)))
ax.set_title(f"Histogram from MCMC with beta={beta}")
ax.set_xlabel("x")
ax.set_ylabel("y")
plt.show()

Now we calculate the population of the minimum energy state (-1.1, 0.5) as a function of 1/$\beta$. We are using the density parameter with the same range, so it really is a probability of occupancy.

In [ ]:
h, x, y = np.histogram2d(*smps.T, bins=30, normed=True, range=((-endpt, endpt), (-endpt, endpt)))
print(f"Left and bottom edges ({x[8]:.2f}, {y[18]}) ")
print(f"Occupation of state {h[8, 18]}")

In [ ]:
occu = []
beta = np.linspace(0.25, 1.0, 25)
for b in beta:
    smps = mcmc(np.array([0.0, 0.0]), 50000, step_size=1.0, beta=b)
    h, x, y = np.histogram2d(*smps.T, bins=30, normed=True, range=((-endpt, endpt), (-endpt, endpt)))
    occu.append(h[8, 18])

In [ ]:
fig, ax = plt.subplots()
ax.plot(1/beta, occu)
ax.set_title("Occupation versus 1/beta")
ax.set_xlabel("1/beta")
ax.set_ylabel("Acceptance Ratio")
plt.show()